In [1]:
# Import required libraries
import pandas as pd
import logging
import sys
from typing import Tuple, Dict
from colorlog import ColoredFormatter
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import tensorflow as tf
from tqdm import tqdm

In [2]:
# Configure the colorful logger
def setup_logger() -> logging.Logger:
    """Set up a colorful logger for the pipeline.
    
    Returns:
        Configured logger instance.
    """
    logger = logging.getLogger("ML_Pipeline")
    logger.setLevel(logging.INFO)
    
    # Define log colors for different levels
    formatter = ColoredFormatter(
        "%(log_color)s%(levelname)-8s%(reset)s | %(log_color)s%(message)s%(reset)s",
        datefmt=None,
        log_colors={
            'DEBUG':    'cyan',
            'INFO':     'white',
            'WARNING':  'yellow',
            'ERROR':    'red',
            'CRITICAL': 'bold_red',
        }
    )
    
    # Stream handler for console output
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    handler.setFormatter(formatter)
    
    logger.addHandler(handler)
    
    return logger

# Initialize the logger
logger = setup_logger()

In [11]:
def load_data() -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Load the AIS and auxiliary datasets.

    Returns:
        A tuple containing the loaded DataFrames:
        (ais_train, ais_test, vessels, ports, schedules).
    """
    try:
        logger.info("Loading datasets.")
        
        # Loading the AIS train and test data with date parsing
        ais_train = pd.read_csv('ais_train.csv', sep='|', parse_dates=['time'], dtype={
            'cog': 'float32', 'sog': 'float32', 'rot': 'int8', 'heading': 'int16', 
            'navstat': 'category', 'etaRaw': 'string', 'latitude': 'float64', 
            'longitude': 'float64', 'vesselId': 'string', 'portId': 'string'
        })

        ais_test = pd.read_csv('ais_test.csv', parse_dates=['time'], dtype={
            'ID': 'string', 'vesselId': 'string', 'scaling_factor': 'float32'
        })

        # Loading vessels, ports, and schedules with specified separators and error handling
        vessels = pd.read_csv('vessels.csv', sep='|', dtype={'vesselId': 'string', 'vesselType': 'string'})
        ports = pd.read_csv('ports.csv', sep='|', dtype={'portId': 'string', 'portName': 'string', 'country': 'string'})
        schedules = pd.read_csv('schedules_to_may_2024.csv', sep='|', on_bad_lines='skip', dtype={
            'scheduleId': 'string', 'vesselId': 'string', 'portId': 'string', 'arrivalTime': 'string'
        })

        logger.info("Datasets loaded successfully.")
        
        # Ensure the function returns the dataframes properly
        return ais_train, ais_test, vessels, ports, schedules

    except FileNotFoundError as e:
        logger.error(f"File not found: {e}")
        raise
    except pd.errors.ParserError as e:
        logger.error(f"Error parsing file: {e}")
        raise
    except Exception as e:
        logger.error(f"An unexpected error occurred: {e}")
        raise

# Load the data
ais_train, ais_test, vessels, ports, schedules = load_data()
# Filter and print rows where the heading is 511
for i in range(len(ais_train)):
    if ais_train[i]['heading'] == 511:
        print(i)

INFO     | Loading datasets.
INFO     | Datasets loaded successfully.


KeyError: 0

In [ ]:
# Check for missing values
missing_values = ais_train.isnull().sum()
print("Missing values per column in ais_train:\n", missing_values)

# Handle outliers
valid_latitude_range = (-90, 90)
valid_longitude_range = (-180, 180)
ais_train = ais_train[(ais_train['latitude'].between(*valid_latitude_range)) & 
                      (ais_train['longitude'].between(*valid_longitude_range))]

max_sog = 50  # Maximum speed threshold
ais_train = ais_train[ais_train['sog'] <= max_sog]

# Normalize numerical features
scaler = StandardScaler()
ais_train[['sog', 'cog']] = scaler.fit_transform(ais_train[['sog', 'cog']])

ais_train['time'] = pd.to_datetime(ais_train['time'])

In [32]:
def prepare_data(
    ais_train: pd.DataFrame,
    vessels: pd.DataFrame, 
    ports: pd.DataFrame, 
    schedules: pd.DataFrame,
    vessel_ids: Dict
) -> Tuple[np.ndarray, np.ndarray, MinMaxScaler, MinMaxScaler]:
    """Prepare the data for the LSTM model, including additional time-based, vessel-specific, and port proximity features.
    
    Args:
        ais_train: DataFrame containing AIS training data.
        
    Returns:
        Tuple containing the feature array (X), target array (y), and the fitted scaler.
    """
    logger.info("Preparing data for the model.")
    
    # Convert the 'time' column to datetime format for feature extraction
    ais_train['time'] = pd.to_datetime(ais_train['time'])
    for col in ['latitude', 'longitude', 'cog', 'sog', 'rot', 'heading', 'etaRaw']:
        ais_train[col] = pd.to_numeric(ais_train[col], errors='coerce')

    # Clip latitude and longitude to their valid ranges
    ais_train['latitude'] = ais_train['latitude'].clip(-90, 90)
    ais_train['longitude'] = ais_train['longitude'].clip(-180, 180)

    print(len(ais_train))

    # List to store the interpolated data points
    interpolated_data = []

    # Loop through each vessel ID
    for vessel_id in tqdm(vessel_ids.keys(), desc="Interpolating Missing Days For Vessels", unit="vessel"):
        # Filter and sort the data for the current vessel ID
        vessel_data = ais_train[ais_train['vesselId'] == vessel_id].sort_values(by='time').reset_index(drop=True)

        # Proceed only if vessel_data is not empty
        if len(vessel_data) > 0:
            # Loop through the sorted data to check time differences
            for i in range(len(vessel_data) - 1):
                current_row = vessel_data.iloc[i]
                next_row = vessel_data.iloc[i + 1]

                time_difference = next_row['time'] - current_row['time']
                
                # Add the current row to the interpolated data list
                interpolated_data.append(current_row)

                # Check if the time difference is greater than 1 day
                if time_difference > pd.Timedelta(days=1):
                    # Calculate the number of missing days
                    num_missing_days = (time_difference.days - 1)

                    # Linearly interpolate values for each missing day
                    for day in range(1, num_missing_days + 1):
                        interpolated_time = current_row['time'] + pd.Timedelta(days=day)
                        
                        # Interpolate all relevant columns
                        interpolated_values = {}
                        for col in ['latitude', 'longitude', 'cog', 'sog', 'rot', 'heading', 'etaRaw']:
                            value_diff = (next_row[col] - current_row[col]) / (num_missing_days + 1)
                            interpolated_values[col] = current_row[col] + value_diff * day
                        
                        # Create a new interpolated data point
                        interpolated_point = current_row.copy()
                        interpolated_point['time'] = interpolated_time
                        interpolated_point['latitude'] = interpolated_values['latitude']
                        interpolated_point['longitude'] = interpolated_values['longitude']
                        # interpolated_point['cog'] = interpolated_values['cog']
                        # interpolated_point['sog'] = interpolated_values['sog']
                        # interpolated_point['rot'] = interpolated_values['rot']
                        # interpolated_point['heading'] = interpolated_values['heading']
                        # interpolated_point['etaRaw'] = interpolated_values['etaRaw']

                        # Copy the values of 'vesselId', 'portId', and 'navstat' directly from the current row
                        interpolated_point['vesselId'] = current_row['vesselId']
                        # interpolated_point['portId'] = current_row['portId']
                        # interpolated_point['navstat'] = current_row['navstat']

                        # Add the interpolated point to the list
                        interpolated_data.append(interpolated_point)

            # Add the last row to the interpolated data list
            interpolated_data.append(vessel_data.iloc[-1])
    # Convert the list of interpolated data back into a DataFrame
    interpolated_df = pd.DataFrame(interpolated_data)

    # Combine the interpolated data with the original ais_train DataFrame
    combined_df = pd.concat([ais_train, interpolated_df]).drop_duplicates().sort_values(by=['vesselId', 'time']).reset_index(drop=True)
    
    ais_train = combined_df

    print(len(ais_train))

    # Extract hour of the day and day of the week as new features
    ais_train['hour_of_day'] = ais_train['time'].dt.hour
    ais_train['day_of_week'] = ais_train['time'].dt.dayofweek

    # Calculate the time elapsed since the first recorded entry for each vessel
    ais_train['time_elapsed'] = (ais_train['time'] - ais_train['time'].min()).dt.total_seconds()
   
    # Map vesselId to its encoded value using vessel_ids dictionary
    ais_train['vesselId_encoded'] = ais_train['vesselId'].map(vessel_ids).fillna(-1).astype(int)

    # Compute the sine and cosine of the course over ground (cog) to represent direction
    ais_train['cog_sin'] = np.sin(np.deg2rad(ais_train['cog']))
    ais_train['cog_cos'] = np.cos(np.deg2rad(ais_train['cog']))

    # Categorize the speed of the vessel
    ais_train['speed_category'] = pd.cut(ais_train['sog'], bins=[-1, 5, 15, np.inf], labels=[0, 1, 2])

    # Merge vessel-specific information into ais_train data
    ais_train = ais_train.merge(vessels[['vesselId', 'maxSpeed', 'length', 'yearBuilt']], on='vesselId', how='left')

    # Fill missing values in vessel-specific data with appropriate defaults
    ais_train['maxSpeed'] = ais_train['maxSpeed'].fillna(ais_train['maxSpeed'].mean())

    # Measure time to create the R-tree index for ports
    start_time = time.time()
    port_idx = index.Index()
    for idx, row in ports.iterrows():
        port_idx.insert(idx, (row['longitude'], row['latitude'], row['longitude'], row['latitude']))
    end_time = time.time()
    logger.info(f"Time to build R-tree index for ports: {end_time - start_time:.2f} seconds")

    # Measure time to calculate the nearest port for each vessel with progress and ETA
    total_vessels = len(ais_train)
    start_time = time.time()
    closest_ports = []

    for _, vessel in tqdm(ais_train.iterrows(), total=total_vessels, desc="Calculating nearest ports", unit="vessel"):
        point = Point(vessel['longitude'], vessel['latitude'])
        nearest_port_idx = list(port_idx.nearest((vessel['longitude'], vessel['latitude'], vessel['longitude'], vessel['latitude']), 1))[0]
        closest_port = ports.iloc[nearest_port_idx]
        distance_to_port = point.distance(Point(closest_port['longitude'], closest_port['latitude']))
        closest_ports.append(distance_to_port)

    end_time = time.time()
    logger.info(f"Time to calculate closest port for all vessels: {end_time - start_time:.2f} seconds for {total_vessels} vessels")

    # Add the calculated distances to the AIS data
    ais_train['distance_to_nearest_port'] = closest_ports

    speed_threshold = 5.0  # knots
    distance_threshold = 1.0  # kilometers

    # Identify if the vessel is anchored
    ais_train['anchored'] = (ais_train['sog'] < speed_threshold) & (ais_train['distance_to_nearest_port'] < distance_threshold)


    # Extract the relevant features, including the new ones
    features = ais_train[['latitude', 'longitude', 'sog', 'day_of_week', 'distance_to_nearest_port', 'anchored', 'vesselId_encoded', 'time_elapsed']].values
    target = ais_train[['latitude', 'longitude']].shift(-1).ffill().values

    # Normalize features
    feature_scaler = MinMaxScaler()
    features_scaled = feature_scaler.fit_transform(features)
   
    # Prepare the features and target values for each vessel separately
    X_list = []
    y_list = []

    # Iterate over each vessel ID in the dataset
    for vessel_id in tqdm(vessel_ids.keys(), total=len(vessel_ids), desc="Finding y 5 days forward", unit="vessel"):
        # Filter data for the current vessel and sort it by time
        vessel_data = ais_train[ais_train['vesselId'] == vessel_id].reset_index(drop=True)

        # Ensure that we have at least 5 days of data for this vessel
        for i in range(len(vessel_data) - 5):
            # Append the feature set (X) for the current day
            X_list.append(vessel_data.iloc[i].values)

            # Append the target set (y) which is the position 5 days later
            y_list.append(vessel_data.iloc[i + 5][['latitude', 'longitude']].values)

    # Convert the collected lists to numpy arrays
    X = np.array(X_list)
    y = np.array(y_list)

    # Reshape for LSTM input: (samples, timesteps, features)
    X = features_scaled.reshape((features_scaled.shape[0], 1, features_scaled.shape[1]))
    
    # Normalize target data (latitude and longitude)
    target_scaler = MinMaxScaler()
    y = target_scaler.fit_transform(target)    
    
    logger.info("Data preparation complete.")
    return X, y, feature_scaler, target_scaler

# Prepare the data
vessel_id_dict = {row["vesselId"]: i for i, row in vessels.iterrows()}
X_train, y_train, feature_scaler, target_scaler = prepare_data(ais_train, vessels, ports, schedules, vessel_id_dict)

INFO     | Preparing data for the model.
1521651


Interpolating Missing Days For Vessels:  11%|█▎           | 75/711 [00:16<01:48,  5.87vessel/s]/var/folders/5q/k66vnm0d2ps_3_nhtc3cx0qr0000gn/T/ipykernel_4719/3821654240.py:61: RuntimeWarning: overflow encountered in scalar subtract
  value_diff = (next_row[col] - current_row[col]) / (num_missing_days + 1)
Interpolating Missing Days For Vessels:  15%|█▊          | 104/711 [00:23<02:19,  4.35vessel/s]/var/folders/5q/k66vnm0d2ps_3_nhtc3cx0qr0000gn/T/ipykernel_4719/3821654240.py:61: RuntimeWarning: overflow encountered in scalar subtract
  value_diff = (next_row[col] - current_row[col]) / (num_missing_days + 1)
Interpolating Missing Days For Vessels:  20%|██▍         | 142/711 [00:29<01:23,  6.80vessel/s]/var/folders/5q/k66vnm0d2ps_3_nhtc3cx0qr0000gn/T/ipykernel_4719/3821654240.py:61: RuntimeWarning: overflow encountered in scalar subtract
  value_diff = (next_row[col] - current_row[col]) / (num_missing_days + 1)
Interpolating Missing Days For Vessels:  20%|██▍         | 143/711 [00:29<01

KeyboardInterrupt: 

In [15]:

def geodesic_loss(y_true, y_pred):
    """Calculate the Haversine distance between true and predicted coordinates.
    
    Args:
        y_true: Tensor of true coordinates (latitude, longitude).
        y_pred: Tensor of predicted coordinates (latitude, longitude).
    
    Returns:
        Tensor representing the geodesic distance (Haversine distance) between the true and predicted points.
    """
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    
    # Split the latitude and longitude into separate tensors
    lat_true, lon_true = tf.split(y_true, num_or_size_splits=2, axis=1)
    lat_pred, lon_pred = tf.split(y_pred, num_or_size_splits=2, axis=1)
    
    # Convert degrees to radians manually
    lat_true = lat_true * tf.constant(np.pi / 180.0)
    lon_true = lon_true * tf.constant(np.pi / 180.0)
    lat_pred = lat_pred * tf.constant(np.pi / 180.0)
    lon_pred = lon_pred * tf.constant(np.pi / 180.0)
    
    # Compute the differences between true and predicted coordinates
    dlat = lat_pred - lat_true
    dlon = lon_pred - lon_true
    
    # Haversine formula
    a = tf.square(tf.sin(dlat / 2)) + tf.cos(lat_true) * tf.cos(lat_pred) * tf.square(tf.sin(dlon / 2))
    c = 2 * tf.atan2(tf.sqrt(a), tf.sqrt(1 - a))
    distance = R * c
    
    return tf.reduce_mean(distance)


In [16]:
def build_model(input_shape: Tuple[int, int]) -> Sequential:
    """Build the LSTM model.
    
    Args:
        input_shape: Shape of the input data (timesteps, features).
        
    Returns:
        Compiled LSTM model.
    """
    logger.info("Building the LSTM model.")
 
    model = Sequential()
    model.add(Input(shape=input_shape))  # Use Input layer to specify the shape
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(units=2))  # Output: latitude and longitude
    model.compile(optimizer='adam', loss=geodesic_loss)
    logger.info("Model built successfully.")
    return model


# Build the model
model = build_model(input_shape=(X_train.shape[1], X_train.shape[2]))

INFO     | Building the LSTM model.
INFO     | Building the LSTM model.
INFO     | Building the LSTM model.
INFO     | Model built successfully.
INFO     | Model built successfully.
INFO     | Model built successfully.


In [17]:
def train_model(model: Sequential, X_train: np.ndarray, y_train: np.ndarray) -> None:
    """Train the LSTM model.
    
    Args:
        model: The LSTM model to train.
        X_train: Training features.
        y_train: Training targets.
    """
    logger.info("Starting model training.")
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2,callbacks=[early_stopping, reduce_lr])
    logger.info("Model training complete.")

# Train the model
train_model(model, X_train, y_train)

INFO     | Starting model training.
INFO     | Starting model training.
INFO     | Starting model training.
Epoch 1/10
38965/38965 ━━━━━━━━━━━━━━━━━━━━ 37s 934us/step - loss: 24.9886 - val_loss: 35.4740 - learning_rate: 0.0010
Epoch 2/10
38965/38965 ━━━━━━━━━━━━━━━━━━━━ 38s 975us/step - loss: 23.6152 - val_loss: 19.8003 - learning_rate: 0.0010
Epoch 3/10
38965/38965 ━━━━━━━━━━━━━━━━━━━━ 43s 1ms/step - loss: 23.5554 - val_loss: 25.7895 - learning_rate: 0.0010
Epoch 4/10
38965/38965 ━━━━━━━━━━━━━━━━━━━━ 40s 1ms/step - loss: 23.5387 - val_loss: 27.1325 - learning_rate: 0.0010
Epoch 5/10
38965/38965 ━━━━━━━━━━━━━━━━━━━━ 46s 1ms/step - loss: 23.4886 - val_loss: 27.8399 - learning_rate: 0.0010
Epoch 6/10
38965/38965 ━━━━━━━━━━━━━━━━━━━━ 43s 1ms/step - loss: 23.4468 - val_loss: 26.8645 - learning_rate: 5.0000e-04
Epoch 7/10
38965/38965 ━━━━━━━━━━━━━━━━━━━━ 46s 1ms/step - loss: 23.4379 - val_loss: 27.5344 - learning_rate: 5.0000e-04
INFO     | Model training complete.
INFO     | Model training

In [18]:
def generate_submission(model: Sequential, ais_test: pd.DataFrame, feature_scaler: MinMaxScaler, target_scaler: MinMaxScaler, vessel_ids: Dict) -> None:
    """Generate a submission file with the predicted vessel positions.
    
    Args:
        model: Trained LSTM model.
        ais_test: DataFrame containing AIS test data.
        feature_scaler: Scaler used to normalize the features.
        target_scaler: Scaler used to normalize the target coordinates.
        vessel_ids: Dictionary to map vessel IDs to numerical indices.
    """
    logger.info("Generating predictions for the test set.")
    
    # Convert the 'time' column to datetime format to handle arithmetic operations
    ais_test['time'] = pd.to_datetime(ais_test['time'], errors='coerce')
    
    # Map vesselId to its encoded value using vessel_ids dictionary
    ais_test['vesselId_encoded'] = ais_test['vesselId'].map(vessel_ids).fillna(-1).astype(int)
   
    # Calculate the time elapsed since the first recorded entry for each vessel
    ais_test['time_elapsed'] = (ais_test['time'] - ais_test['time'].min()).dt.total_seconds()

    # Extract the relevant features for the test data
    test_features = ais_test[['vesselId_encoded', 'time_elapsed']].values

    # Since the test data only has one feature, we need to adjust the input shape to match the model's expectation
    num_train_features = feature_scaler.n_features_in_
    test_features_padded = np.zeros((test_features.shape[0], num_train_features))
    test_features_padded[:, :test_features.shape[1]] = test_features

    # Normalize the padded test features to match the training data scale
    test_features_scaled = feature_scaler.transform(test_features_padded)
    X_test = test_features_scaled.reshape((test_features_scaled.shape[0], 1, test_features_scaled.shape[1]))
    
    # Make predictions using the model
    predictions = model.predict(X_test)
    
    # Inverse transform the predictions using the target scaler
    predictions = target_scaler.inverse_transform(predictions)
    
    # Create the submission DataFrame in the required format
    submission = pd.DataFrame({
        'ID': ais_test['ID'],
        'longitude_predicted': predictions[:, 1],
        'latitude_predicted': predictions[:, 0]
    })
   
    # Ensure that the submission file has exactly 51739 rows as required
    assert submission.shape[0] == 51739, "The submission file must have exactly 51739 rows."
    
    # Save the predictions to submission.csv
    submission.to_csv('submission.csv', index=False)
    logger.info("Submission file saved as submission.csv.")


# Generate the submission file
generate_submission(model, ais_test, feature_scaler, target_scaler, vessel_id_dict)

INFO     | Generating predictions for the test set.
INFO     | Generating predictions for the test set.
INFO     | Generating predictions for the test set.
1617/1617 ━━━━━━━━━━━━━━━━━━━━ 1s 438us/step
INFO     | Submission file saved as submission.csv.
INFO     | Submission file saved as submission.csv.
INFO     | Submission file saved as submission.csv.


In [19]:
# Load the CSV file
df = pd.read_csv('submission.csv')

# Calculate the maximum and minimum values for longitude_predicted and latitude_predicted
longitude_max = df['longitude_predicted'].max()
longitude_min = df['longitude_predicted'].min()
latitude_max = df['latitude_predicted'].max()
latitude_min = df['latitude_predicted'].min()

# Print the results
print(f"Maximum Longitude Predicted: {longitude_max}")
print(f"Minimum Longitude Predicted: {longitude_min}")
print(f"Maximum Latitude Predicted: {latitude_max}")
print(f"Minimum Latitude Predicted: {latitude_min}")

Maximum Longitude Predicted: 28.37741
Minimum Longitude Predicted: 3.8418357
Maximum Latitude Predicted: 56.508556
Minimum Latitude Predicted: 38.977943


In [12]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from rtree import index
from shapely.geometry import Point
from tqdm import tqdm
import time
import logging

# Initialize logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Load the data
ais_train = pd.read_csv('ais_train.csv', sep='|')
ais_test = pd.read_csv('ais_test.csv')
ports = pd.read_csv('ports.csv', sep='|')  # Assuming you have a ports.csv file to load port data

# Measure time to create the R-tree index for ports
start_time = time.time()
port_idx = index.Index()
for idx, row in ports.iterrows():
    port_idx.insert(idx, (row['longitude'], row['latitude'], row['longitude'], row['latitude']))
end_time = time.time()
logger.info(f"Time to build R-tree index for ports: {end_time - start_time:.2f} seconds")

# Measure time to calculate the nearest port for each vessel with progress and ETA
total_vessels = len(ais_train)
start_time = time.time()
closest_ports = []

for _, vessel in tqdm(ais_train.iterrows(), total=total_vessels, desc="Calculating nearest ports", unit="vessel"):
    point = Point(vessel['longitude'], vessel['latitude'])
    nearest_port_idx = list(port_idx.nearest((vessel['longitude'], vessel['latitude'], vessel['longitude'], vessel['latitude']), 1))[0]
    closest_port = ports.iloc[nearest_port_idx]
    distance_to_port = point.distance(Point(closest_port['longitude'], closest_port['latitude']))
    closest_ports.append(distance_to_port)

end_time = time.time()
logger.info(f"Time to calculate closest port for all vessels: {end_time - start_time:.2f} seconds for {total_vessels} vessels")

# Add the calculated distances to the AIS data
ais_train['distance_to_nearest_port'] = closest_ports

speed_threshold = 5.0  # knots
distance_threshold = 1.0  # kilometers

# Identify if the vessel is anchored
ais_train['anchored'] = (ais_train['sog'] < speed_threshold) & (ais_train['distance_to_nearest_port'] < distance_threshold)

# Convert 'time' column to datetime and extract useful features like hour, minute, and total time elapsed
ais_train['time'] = pd.to_datetime(ais_train['time'], errors='coerce')
ais_train['hour'] = ais_train['time'].dt.hour
ais_train['minute'] = ais_train['time'].dt.minute

ais_test['time'] = pd.to_datetime(ais_test['time'], errors='coerce')
ais_test['hour'] = ais_test['time'].dt.hour
ais_test['minute'] = ais_test['time'].dt.minute

# Calculate the total time elapsed in seconds from the earliest time in the dataset
reference_time = ais_train['time'].min()
ais_train['time_elapsed'] = (ais_train['time'] - reference_time).dt.total_seconds()
ais_test['time_elapsed'] = (ais_test['time'] - reference_time).dt.total_seconds()

# Fill missing values in 'hour', 'minute', and 'time_elapsed' with a placeholder (e.g., -1) to handle NaN values
ais_train[['hour', 'minute', 'time_elapsed']] = ais_train[['hour', 'minute', 'time_elapsed']].fillna(-1)
ais_test[['hour', 'minute', 'time_elapsed']] = ais_test[['hour', 'minute', 'time_elapsed']].fillna(-1)

# Encode categorical features like 'vesselId' using Label Encoding
le = LabelEncoder()
ais_train['vesselId_encoded'] = le.fit_transform(ais_train['vesselId'].astype(str))
ais_test['vesselId_encoded'] = le.transform(ais_test['vesselId'].astype(str))

# Define numeric columns for training separately, including the new time_elapsed feature
train_numeric_columns = ['cog', 'sog', 'rot', 'heading', 'navstat', 'etaRaw', 'latitude', 'longitude', 'vesselId_encoded', 'time_elapsed', 'distance_to_nearest_port', 'anchored']

# Convert training columns to numeric, coercing errors to NaN
ais_train[train_numeric_columns] = ais_train[train_numeric_columns].apply(pd.to_numeric, errors='coerce')

# Fill any remaining NaN values in the dataset with appropriate defaults
ais_train[train_numeric_columns] = ais_train[train_numeric_columns].fillna(0)

# Extract features and targets for the first model using the full training set
X_train_full = ais_train[train_numeric_columns].values
y_train_full = ais_train[['latitude', 'longitude']].values

# Normalize/Scale the full training set
scaler_X_train_full = StandardScaler()
X_train_full = scaler_X_train_full.fit_transform(X_train_full)

scaler_y_full = StandardScaler()
y_train_full = scaler_y_full.fit_transform(y_train_full)

# Reshape the data for LSTM input (samples, timesteps, features)
X_train_full = X_train_full.reshape(X_train_full.shape[0], 1, X_train_full.shape[1])

# Build and train the first model on the full training set
input_full = Input(shape=(1, X_train_full.shape[2]), name='full_input')
lstm_full = LSTM(64, return_sequences=False)(input_full)
dense_full = Dense(32, activation='relu')(lstm_full)
output_full = Dense(2, activation='linear', name='coordinates_output')(dense_full)

model_full = Model(inputs=input_full, outputs=output_full)
model_full.compile(optimizer='adam', loss='mse')
model_full.fit(X_train_full, y_train_full, epochs=10, batch_size=32)

# Make predictions on the full training set using the first model
predicted_coords = model_full.predict(X_train_full)
predicted_coords = scaler_y_full.inverse_transform(predicted_coords)  # Reverse scaling

# Prepare input data for the second model using vesselId, hour, minute, and time_elapsed features
X_train_second_model = ais_train[['vesselId_encoded', 'time_elapsed']].values
y_train_second_model = predicted_coords  # Use the predicted coordinates as labels

# Normalize/Scale the input data for the second model
scaler_X_second = StandardScaler()
X_train_second_model = scaler_X_second.fit_transform(X_train_second_model)

# Reshape the data for LSTM input (samples, timesteps, features)
X_train_second_model = X_train_second_model.reshape(X_train_second_model.shape[0], 1, X_train_second_model.shape[1])

# Build and train the second model on vesselId, time, and time_elapsed
input_second = Input(shape=(1, X_train_second_model.shape[2]), name='second_input')
lstm_second = LSTM(64, return_sequences=False)(input_second)
dense_second = Dense(32, activation='relu')(lstm_second)
output_second = Dense(2, activation='linear', name='coordinates_predicted')(dense_second)

model_second = Model(inputs=input_second, outputs=output_second)
model_second.compile(optimizer='adam', loss='mse')
model_second.fit(X_train_second_model, y_train_second_model, epochs=10, batch_size=32)

# Make predictions on the test set using the second model
X_test_subset = ais_test[['vesselId_encoded', 'time_elapsed']].values
X_test_subset = scaler_X_second.transform(X_test_subset)
X_test_subset = X_test_subset.reshape(X_test_subset.shape[0], 1, X_test_subset.shape[1])

test_predictions = model_second.predict(X_test_subset)

# Reverse the scaling to get the original longitude and latitude values
test_predictions = scaler_y_full.inverse_transform(test_predictions)

# Generate the submission file with the predicted vessel positions
submission = pd.DataFrame({
    'ID': ais_test['ID'],
    'longitude_predicted': test_predictions[:, 1],
    'latitude_predicted': test_predictions[:, 0]
})

# Ensure that the submission file has exactly 51739 rows as required
assert submission.shape[0] == 51739, "The submission file must have exactly 51739 rows."

# Save the submission to a CSV file
submission.to_csv('submission.csv', index=False)

print("Submission file created successfully with 51739 rows.")

Calculating nearest ports: 100%|██████████████████████████████████████████████████████████████████████████████| 1522065/1522065 [02:12<00:00, 11528.05vessel/s]


Epoch 1/10
47565/47565 ━━━━━━━━━━━━━━━━━━━━ 27s 553us/step - loss: 0.0093
Epoch 2/10
47565/47565 ━━━━━━━━━━━━━━━━━━━━ 26s 552us/step - loss: 4.3165e-05
Epoch 3/10
47565/47565 ━━━━━━━━━━━━━━━━━━━━ 26s 552us/step - loss: 2.9144e-05
Epoch 4/10
47565/47565 ━━━━━━━━━━━━━━━━━━━━ 26s 555us/step - loss: 2.2083e-05
Epoch 5/10
47565/47565 ━━━━━━━━━━━━━━━━━━━━ 26s 550us/step - loss: 1.8692e-05
Epoch 6/10
47565/47565 ━━━━━━━━━━━━━━━━━━━━ 27s 558us/step - loss: 1.6073e-05
Epoch 7/10
47565/47565 ━━━━━━━━━━━━━━━━━━━━ 27s 560us/step - loss: 1.4307e-05
Epoch 8/10
47565/47565 ━━━━━━━━━━━━━━━━━━━━ 27s 561us/step - loss: 1.2737e-05
Epoch 9/10
47565/47565 ━━━━━━━━━━━━━━━━━━━━ 28s 582us/step - loss: 1.1609e-05
Epoch 10/10
47565/47565 ━━━━━━━━━━━━━━━━━━━━ 27s 558us/step - loss: 1.1753e-05
47565/47565 ━━━━━━━━━━━━━━━━━━━━ 15s 305us/step
Epoch 1/10
47565/47565 ━━━━━━━━━━━━━━━━━━━━ 27s 549us/step - loss: 2623.6301
Epoch 2/10
47565/47565 ━━━━━━━━━━━━━━━━━━━━ 719s 15ms/step - loss: 2595.5369
Epoch 3/10
47565/4756